### Train CNN Model 3 Classes

In [14]:
import os
import json
import numpy as np
import librosa
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, regularizers
from imblearn.over_sampling import SMOTE

DATA_PATH = "/Users/danielporras/Musica/musica_analysis/models/data.json_3_classes"

def extract_features(file_path):
    # Your feature extraction code remains the same here...
    pass

def add_noise(X_train):
    noise_factor = 0.005 * np.random.randn(*X_train.shape)
    X_train_noisy = X_train + noise_factor
    return X_train_noisy

def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["features"])
    y = np.array(data["labels"])

    # Standardize features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Apply SMOTE
    smote = SMOTE()
    X_resampled, y_resampled = smote.fit_resample(X, y)

    return X_resampled, y_resampled, scaler

def prepare_datasets(test_size, validation_size):
    X, y, scaler = load_data(DATA_PATH)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # Add noise to the training set
    X_train = add_noise(X_train)

    # Reshape for Conv1D
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test, scaler

def build_model(input_shape):
    model = models.Sequential()

    # 1D Convolutional layers
    model.add(layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv1D(128, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(pool_size=2, padding='same'))
    model.add(layers.BatchNormalization())

    # Recurrent layer
    model.add(layers.LSTM(64, return_sequences=False))

    # Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.001)))
    model.add(layers.Dropout(0.3))
    model.add(layers.Dense(3, activation='softmax'))  # Adjusted to 3 classes

    return model

if __name__ == "__main__":
    # Load the data
    X_train, X_validation, X_test, y_train, y_validation, y_test, scaler = prepare_datasets(0.25, 0.2)

    input_shape = (X_train.shape[1], X_train.shape[2])

    # Build the model
    model = build_model(input_shape)

    # Compile the model
    optimiser = tf.keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(optimizer=optimiser,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_validation, y_validation),
                        epochs=100, batch_size=32)

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

    # Compute predictions
    y_prob = model.predict(X)
    y_pred = np.argmax(y_prob, axis=1)

    # Flatten y back to 1D
    y = y.flatten()

    # Compute predictions
    y_prob = model.predict(X)
    y_pred = np.argmax(y_prob, axis=1)

    # Create confusion matrix
    cm = confusion_matrix(y, y_pred, labels=[0, 1, 2])

    # Display confusion matrix
    cm_df = pd.DataFrame(cm, index=[f'True {i}' for i in range(3)], columns=[f'Predicted {i}' for i in range(3)])
    print("Confusion Matrix:")
    print(cm_df)

    # Compute precision and F1 score
    precision = precision_score(y, y_pred, average='weighted')
    print(f'Precision: {precision}')

    f1 = f1_score(y, y_pred, average='weighted')
    print(f'F1 Score: {f1}')


    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

Epoch 1/100
11/11 [==============================] - 1s 26ms/step - loss: 1.1730 - accuracy: 0.4713 - val_loss: 1.2071 - val_accuracy: 0.3182
Epoch 2/100
11/11 [==============================] - 0s 8ms/step - loss: 1.0329 - accuracy: 0.7500 - val_loss: 1.1955 - val_accuracy: 0.4205
Epoch 3/100
11/11 [==============================] - 0s 7ms/step - loss: 0.8814 - accuracy: 0.8190 - val_loss: 1.1774 - val_accuracy: 0.4432
Epoch 4/100
11/11 [==============================] - 0s 7ms/step - loss: 0.6976 - accuracy: 0.8649 - val_loss: 1.1588 - val_accuracy: 0.4091
Epoch 5/100
11/11 [==============================] - 0s 7ms/step - loss: 0.5359 - accuracy: 0.9080 - val_loss: 1.1426 - val_accuracy: 0.3636
Epoch 6/100
11/11 [==============================] - 0s 7ms/step - loss: 0.4007 - accuracy: 0.9339 - val_loss: 1.1417 - val_accuracy: 0.3977
Epoch 7/100
11/11 [==============================] - 0s 8ms/step - loss: 0.2904 - accuracy: 0.9655 - val_loss: 1.1669 - val_accuracy: 0.3864
Epoch 8/100


### json_3_classes_2

In [19]:
import os
import json
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers, models, regularizers
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, f1_score, classification_report

DATA_PATH = "/Users/danielporras/Musica/musica_analysis/models/data.json_3_classes_2"


def add_noise(X_train):
    noise_factor = 0.005 * np.random.randn(*X_train.shape)
    X_train_noisy = X_train + noise_factor
    return X_train_noisy

def load_data(data_path):
    with open(data_path, "r") as fp:
        data = json.load(fp)

    X = np.array(data["features"])
    y = np.array(data["labels"])

    # Reshape for Conv2D: Add a channel dimension
    X = X.reshape(X.shape[0], 40, 174, 1)  # Adjust shape to match the CNN input

    # Standardize features
    # It's crucial to fit the scaler on the training set only to avoid data leakage
    # Splitting data first
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    scaler = StandardScaler().fit(X_train.reshape(X_train.shape[0], -1))
    
    # Transform and then reshape back to original shape for both train and test sets
    X_train = scaler.transform(X_train.reshape(X_train.shape[0], -1)).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(X_test.shape[0], -1)).reshape(X_test.shape)

    # Apply SMOTE on flattened X_train to avoid dimensionality issues
    smote = SMOTE()
    X_train_flat, y_train_resampled = smote.fit_resample(X_train.reshape(X_train.shape[0], -1), y_train)
    X_train_resampled = X_train_flat.reshape(-1, 40, 174, 1)

    return X_train_resampled, X_test, y_train_resampled, y_test, scaler

def build_model(input_shape):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dropout(0.5),
        layers.Dense(3, activation='softmax')  # Adjusted to 3 classes
    ])

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

if __name__ == "__main__":
    X_train, X_test, y_train, y_test, scaler = load_data(DATA_PATH)
    input_shape = X_train.shape[1:]  # Input shape for Conv2D

    model = build_model(input_shape)
    history = model.fit(X_train, y_train, validation_split=0.2, epochs=30, batch_size=32)

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

        # Predict the probabilities for the test set
    predictions_prob = model.predict(X_test)
    # Convert these probabilities to class labels
    predictions = np.argmax(predictions_prob, axis=1)

    # Calculate and display the classification report
    print("Classification Report:")
    print(classification_report(y_test, predictions, target_names=['happy_exciting', 'heavy_rock', 'sad_reflective']))

    # Calculate and display the confusion matrix
    print("Confusion Matrix:")
    cm = confusion_matrix(y_test, predictions)
    print(cm)

    # Save the model
    model.save('/Users/danielporras/Musica/music_x/best_model.h8')

    # Use the trained scaler to transform new data for predictions
    # X_new_transformed = scaler.transform(X_new)


Epoch 1/30
11/11 [==============================] - 1s 55ms/step - loss: 1.7545 - accuracy: 0.4665 - val_loss: 1.1718 - val_accuracy: 0.7791
Epoch 2/30
11/11 [==============================] - 1s 49ms/step - loss: 1.0785 - accuracy: 0.6356 - val_loss: 0.8447 - val_accuracy: 0.7442
Epoch 3/30
11/11 [==============================] - 1s 48ms/step - loss: 0.8087 - accuracy: 0.7988 - val_loss: 0.6896 - val_accuracy: 0.9070
Epoch 4/30
11/11 [==============================] - 1s 51ms/step - loss: 0.5496 - accuracy: 0.9009 - val_loss: 0.4863 - val_accuracy: 0.9419
Epoch 5/30
11/11 [==============================] - 1s 52ms/step - loss: 0.4236 - accuracy: 0.9534 - val_loss: 0.3278 - val_accuracy: 0.9767
Epoch 6/30
11/11 [==============================] - 1s 47ms/step - loss: 0.3203 - accuracy: 0.9767 - val_loss: 0.2579 - val_accuracy: 1.0000
Epoch 7/30
11/11 [==============================] - 1s 49ms/step - loss: 0.2642 - accuracy: 0.9913 - val_loss: 0.2317 - val_accuracy: 1.0000
Epoch 8/30
11

INFO:tensorflow:Assets written to: /Users/danielporras/Musica/music_x/best_model.h8/assets
